In [1]:
#IMPORTS
#Install+Import Packages
!pip install mechanize
import random
import re
import mechanize
import numpy as np
from random import *
import matplotlib.pyplot as plt
!pip install Bio
from Bio import SeqIO
from Bio.Seq import Seq
from collections import Counter 
from Bio.Cluster import kcluster
import statistics
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import difflib
import seaborn as sns; sns.set_theme()
from collections import defaultdict
import collections
from mpl_toolkits.mplot3d import Axes3D
from pylab import *
from collections import OrderedDict
import matplotlib.pyplot as plt
#!pip install ipython-autotime

In [2]:
#Upload Data and Define Parameters

file_type = '.fasta'

aa_sequences = open('/Users/zoeweiss/Desktop/Desktop/lab(s)/szostak/spring 2022/aminoacylation/Str_AA_Merged_lowqual.fasta').readlines()
rna_only_sequences = open('/Users/zoeweiss/Desktop/Desktop/lab(s)/szostak/spring 2022/aminoacylation/Str-NaOH_Merged_lowqual.fasta').readlines()

#Take the reverse complement? Enter 'yes' or 'no'
rc = 'no'

#Quality Cutoff - what fraction of positions should have <1% error probability?
quality_cutoff = 0.9

forward_motif = 'GUUCAGAGUUCUACAGUCCGACGAUCAAA'.replace('U', 'T')

forward_motif



'GTTCAGAGTTCTACAGTCCGACGATCAAA'

In [3]:
def preprocess_seqs(list_of_seqs):

    if file_type == '.fastq':
        seqs = list_of_seqs[1::4]
        qualities = list_of_seqs[3::4]
        print('Total seqs: '+str(len(seqs)))

        #Filter by quality
        high_quality = []
        acceptable_quals_1_percent_error = [5,6,7,8,9,':',';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K']
        for seq in range(len(qualities)):
            overlap=0
            for char in list(set(qualities[seq]).intersection(acceptable_quals_1_percent_error)):
                overlap+=qualities[seq].count(char)
            if overlap/len(qualities[1])>quality_cutoff:
                high_quality.append(seqs[seq].strip("\n"))
        print('High quality seqs: '+str(len(high_quality)))
    
    if file_type == '.fasta':
        high_quality = list_of_seqs[1::2]
        print('Total seqs: '+str(len(high_quality)))
    
    #Take RC if desired
    pre_trim = []
    if rc == 'yes':
        rc_seqs = []
        for sequence in high_quality:
            rc_seqs.append(str(Seq(sequence).reverse_complement()).strip('\n'))
        pre_trim = rc_seqs
    else:
        for sequence in high_quality:
            pre_trim.append(str(Seq(sequence)).strip('\n'))
    if rc == 'yes':
        print('Reverse Comp = '+str(rc)) 

    #Trim to keep only seqs with constant region
    trimmed = []
    for seq in pre_trim:
        if forward_motif in seq:
            next_40 = seq[seq.index(forward_motif)+len(forward_motif):seq.index(forward_motif)+len(forward_motif)+40]
            if len(next_40)==40:
                trimmed.append(next_40)
    print('Seqs after trimming: '+str(len(trimmed))) 
    
    #All processed seqs
    processed_seqs = trimmed
    #Unique seqs
    processed_seqs_unique = list(set(trimmed))
    
    return processed_seqs, processed_seqs_unique
    

#Function to get secondary structure on RNA Fold
def get_energy_ss(seq):
  url = "http://rna.tbi.univie.ac.at//cgi-bin/RNAWebSuite/RNAfold.cgi"
  br = mechanize.Browser()
  br.set_handle_robots(False) # ignore robots
  br.open(url)
  br.select_form(name="form")
  br["SCREEN"] = seq
  res = br.submit()
  content = res.read()
  energy = int(re.sub("[^0-9]", "", str(content).split('thermodynamic ensemble is')[1].split('kcal/mol')[0]))
  structure = str(content).split('STRUCTURE=')[1].split('target')[0].split('"')[0]
  return energy, structure



In [4]:
#AA CONTROL
#STATISTICS (note: this takes a few mins per round)
aa_ctrl_processed_seqs, aa_ctrl_processed_seqs_unique = preprocess_seqs(aa_sequences)
aa_ctrl_top_seqs = Counter(aa_ctrl_processed_seqs).most_common(100)
print(aa_ctrl_top_seqs[:10])

Total seqs: 3006527
Seqs after trimming: 2378029
[('AAGATCGGAAGAGCACACGTCTGAACTCCAGTCACATCAC', 1675), ('AAAGATCGGAAGAGCACACGTCTGAACTCCAGTCACATCA', 619), ('GGAAAAGATCGGAAGAGCACACGTCTGAACTCCAGTCACA', 445), ('GAAAAGATCGGAAGAGCACACGTCTGAACTCCAGTCACAT', 402), ('GGGAAAAGATCGGAAGAGCACACGTCTGAACTCCAGTCAC', 357), ('AAAAGATCGGAAGAGCACACGTCTGAACTCCAGTCACATC', 328), ('GGGGAAAAGATCGGAAGAGCACACGTCTGAACTCCAGTCA', 202), ('GGTAAAAGATCGGAAGAGCACACGTCTGAACTCCAGTCAC', 177), ('GTAAAAGATCGGAAGAGCACACGTCTGAACTCCAGTCACA', 169), ('GCAAAAGATCGGAAGAGCACACGTCTGAACTCCAGTCACA', 157)]


In [5]:
#RNA CONTROL
#STATISTICS (note: this takes a few mins per round)
rna_ctrl_processed_seqs, rna_ctrl_processed_seqs_unique = preprocess_seqs(rna_only_sequences)
rna_ctrl_top_seqs = Counter(rna_ctrl_processed_seqs).most_common(100)
print(rna_ctrl_top_seqs[:10])



Total seqs: 3449734
Seqs after trimming: 2682013
[('AAGATCGGAAGAGCACACGTCTGAACTCCAGTCACCGATG', 6080), ('AAAGATCGGAAGAGCACACGTCTGAACTCCAGTCACCGAT', 2036), ('GAGATCGGAAGAGCACACGTCTGAACTCCAGTCACCGATG', 857), ('AAAAGATCGGAAGAGCACACGTCTGAACTCCAGTCACCGA', 798), ('GAAAAGATCGGAAGAGCACACGTCTGAACTCCAGTCACCG', 604), ('TAGATCGGAAGAGCACACGTCTGAACTCCAGTCACCGATG', 583), ('GGAAAAGATCGGAAGAGCACACGTCTGAACTCCAGTCACC', 470), ('GAAGATCGGAAGAGCACACGTCTGAACTCCAGTCACCGAT', 377), ('TAAAAGATCGGAAGAGCACACGTCTGAACTCCAGTCACCG', 367), ('GGGAAAAGATCGGAAGAGCACACGTCTGAACTCCAGTCAC', 336)]


In [7]:
#AA NOT RNA 
#STATISTICS (note: this takes a few mins per round)

aa_rna_overlap = list(set(aa_ctrl_processed_seqs) & set(rna_ctrl_processed_seqs))

aa_not_rna_seqs = aa_ctrl_processed_seqs
counter = 0
for seq in range(len(aa_rna_overlap)):
    counter+=1
    if counter%100==0:
        print(str(counter)+'/'+str(len(aa_rna_overlap)))
        
    aa_not_rna_seqs.remove(aa_rna_overlap[seq])

print('Seqs in AA but not RNA: '+str(round(len(aa_not_rna_seqs))))

aa_not_rna_top_seqs = Counter(aa_not_rna_seqs).most_common(100)
print(aa_not_rna_top_seqs[:10])


100/3291
200/3291
300/3291
400/3291
500/3291
600/3291
700/3291
800/3291
900/3291
1000/3291
1100/3291
1200/3291
1300/3291
1400/3291
1500/3291
1600/3291
1700/3291
1800/3291
1900/3291
2000/3291
2100/3291
2200/3291
2300/3291
2400/3291
2500/3291
2600/3291
2700/3291
2800/3291
2900/3291
3000/3291
3100/3291
3200/3291
Seqs in AA but not RNA: 2366312
[('AAGATCGGAAGAGCACACGTCTGAACTCCAGTCACATCAC', 1675), ('AAAGATCGGAAGAGCACACGTCTGAACTCCAGTCACATCA', 619), ('GGAAAAGATCGGAAGAGCACACGTCTGAACTCCAGTCACA', 445), ('GAAAAGATCGGAAGAGCACACGTCTGAACTCCAGTCACAT', 402), ('GGGAAAAGATCGGAAGAGCACACGTCTGAACTCCAGTCAC', 355), ('AAAAGATCGGAAGAGCACACGTCTGAACTCCAGTCACATC', 328), ('GGGGAAAAGATCGGAAGAGCACACGTCTGAACTCCAGTCA', 200), ('GGTAAAAGATCGGAAGAGCACACGTCTGAACTCCAGTCAC', 175), ('GTAAAAGATCGGAAGAGCACACGTCTGAACTCCAGTCACA', 169), ('GCAAAAGATCGGAAGAGCACACGTCTGAACTCCAGTCACA', 157)]


In [ ]:
#Get the SS of the top sequences
ss_of_top_seqs=[]
for seq in aa_not_rna_top_seqs[:100]:
  if aa_not_rna_top_seqs.index(seq)%2==0:
    print(aa_not_rna_top_seqs.index(seq))
  ss_of_top_seqs.append(get_energy_ss(seq[0])[1])

0
2
4
6
8
10
12
14
16
18
20


In [ ]:
#Cluster by secondary structure
k = 3
matrix = np.asarray([np.fromstring(str(s), dtype=np.uint8) for s in ss_of_top_seqs]);
kmeans = KMeans(init="random", n_clusters=k,n_init=10,max_iter=300,random_state=42)
kmeans.fit(matrix)
labels = kmeans.labels_[:]

all_clusters = []
for cluster_num in range(0, k):
  seqs_in_cluster = []
  for seq in range(len(labels)):
    if labels[seq] == cluster_num:
      seqs_in_cluster.append(ss_of_top_seqs[seq])
  all_clusters.append(seqs_in_cluster)
all_clusters